EPITA 2022 MLRF practice_01-04_color-histogram v2022-03-30_095240 by Joseph CHAZALON

<div style="overflow: auto; padding: 10px; margin: 10px 0px">
<img alt="Creative Commons License" src='img/CC-BY-4.0.png' style='float: left; margin-right: 20px'>
    
This work is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).
</div>

# Practice 1 part 4: (Indexed) color histograms as global descriptors

The principle is very simple and illustrated below: from a set of pixels we count how many times each color appears, and we build an histogram of the frequencies (as opposed to raw counts, because we normalize the values) of occurrences of colors.

Here are some examples of the histograms we can compute from some bubbles:
![hist1](img/hist1.png)
![hist2](img/hist2.png)
![hist3](img/hist3.png)

Using such descriptors, we can very easily group similar bubbles with a reasonable confidence.

This part contains the following steps:
1. Color quantization: reduce the colors of the bubbles.
2. Compute the color histogram of each bubble.
3. Compute the distance matrix between each bubble, using its color histogram.
4. Visualize the bubbles in an interesting way using hierarchical clustering.

## 1. Color quantization
It is hard to compare the full RGB histogram of an image (a bubble) with the histogram of another, so we will first **reduce the number of colors** used to represent each image.


Color quantization is a practical application of [vector quantization](https://en.wikipedia.org/wiki/Vector_quantization) where each color is replaced by the closest color in a pre-defined palette.


We have two options here:
1. build a palette manually
2. use a clustering technique to build it automatically

We will use **K-Means clustering** to discover a reduced set of representative colors.

<div style="overflow: auto; border-style: solid; border-color: red; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="stop" src='img/stop.png' style='float: left; margin-right: 20px'>
    
**If you do not manage to build the palette automatically, then build your codebook manually and convert the color of the images using [`scipy.cluster.vq.vq`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.vq.vq.html).**
</div>

### 1.1. Sample some pixels
Because K-Means is a costly algorithm, we will first sample our pixels (viewing them as plain 3-dimensional vectors) to avoid filling up our memory during KMeans fitting.

We will use the base image to facilitate the sampling, because otherwise we would have to select pixels from every bubble image and merge the results.

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Use the large (200 DPI) poster image WITH ITS MASK to sample some pixels. 5000 is a good number of samples.**

*Tips:*
- The code for loading the image is provided below to save you time.
- `poster[poster_mask]` returns a set of valid pixels.
- You can use [`np.random.choice`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) to select **random pixems** from the set of pixels belonging to the bubbles.
- The overall idea is the run 2 "select" operations on the original image: 1) select the pixels which belong to bubbles (i.e. not in the background) and 2) select some random pixels among these.
</div>

In [ ]:
# deactivate buggy jupyter completion
%config Completer.use_jedi = False

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [ ]:
def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

In [ ]:
# TODO
PATH_TO_RESOURCES = "../resources"  # FIXME set this to the path of the twinit resource directory
reference_img = load_image(PATH_TO_RESOURCES + "/dobble_ref.png")

### 1.4. Load and convert all the bubbles
We can now load and convert all the bubbles. We need to:
1. load them
2. convert them to the same color space as use during training (RGB here)
3. project their colors using the previous method

We will need to ignore the area where all the pixels are black `(0,0,0)`, because they do not belong to the bubble and it may change their value.

In [ ]:
# define a function to compute the mask of a bubble
def symbol2mask(symbol):
    return np.sum(symbol, axis=2) < 255 * 3

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Now load all the bubbles and convert them to RGB (or any color space your choosed).**
</div>

In [ ]:
# Some Jupyter magic to help you
symbol_files = !ls $PATH_TO_RESOURCES/**/c*.png  | sort
symbol_files

In [ ]:
# TODO load all the bubbles and convert them to RGB
# `bubbles` is a list of np.array elements (bubble images in RGB format)
symbols = [cv2.cvtColor(cv2.imread(symbol_file), cv2.COLOR_BGR2RGB) for symbol_file in symbol_files]
len(symbols)

In [ ]:
symbols_masks = [symbol2mask(symbol) for symbol in symbols]
len(symbols_masks)

<img alt="warning" src='img/warning.png' style='float: left; margin-right: 20px'>

**WARNING: All our images should be in RGB format in this session! Convert them when loading them to avoid mistakes.**

In [ ]:
# let us display the images
plt.figure(figsize=(8,16 * 10))
for ii in range(57):
    plt.subplot(57,2,1+2*ii)
    plt.imshow(symbols[ii * 5])
    plt.axis("off")
    if ii == 0:
        plt.title("Original")
    plt.subplot(57,2,2+2*ii)
    plt.imshow(symbols_masks[ii * 5])
    plt.axis("off")
    if ii == 0:
        plt.title("Recolored")

In [ ]:
sample_pixels = symbols[0][symbols_masks[0]]
for i in range(1, len(symbols)):
    sample_pixels = np.concatenate((sample_pixels, symbols[i][symbols_masks[i]]), axis=0)

sample_pixels.shape, sample_pixels.dtype



### 1.2. K-Means clustering stage 1/2: fitting
We are now ready to perform K-Means clustering.

In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Use K-Means clustering to compute the color palette. Use a small value like 7 for the number of target colors.**

Tip:
- Set the `random_state` parameter to some fixed value to ensure the reproducibility of your results.
- Use the `fit` method to compute cluster centers.
- The cluster centers will be available with `kmeans.cluster_centers_`.
</div>

<img alt="warning" src='img/warning.png' style='float: left; margin-right: 20px'>

**WARNING: K-Means is a RAM-hungry algorithm. Save your work regularly (and start now)!**

In [ ]:
def find_and_plot_cluster_centers(clusters_nb):
    # Train kmeans
    kmeans = MiniBatchKMeans(clusters_nb, random_state=42)
    kmeans.fit(sample_pixels)
    
    # Create the LUT
    color_lut = np.uint8(kmeans.cluster_centers_)
    
    # Sort it by hue
    hue_lut = cv2.cvtColor(color_lut.reshape(1,color_lut.shape[0], color_lut.shape[1]), cv2.COLOR_BGR2HSV)[:, :, 0]
    indices = hue_lut.argsort(axis=None)
    color_lut_sorted = color_lut[indices]
    
    # Plot it
    plt.bar(np.arange(len(color_lut)), 
        np.ones(len(color_lut)), 
        color=color_lut_sorted/255)
    
    return kmeans, color_lut

In [ ]:
%%time
# let us display the images
plt.figure(figsize=(8,20))
max_clusters_nb = 30
variances = np.zeros(max_clusters_nb - 1)

for ii in range(1, max_clusters_nb):
    plt.subplot(max_clusters_nb, 1, ii)
    
    kmeans, color_lut = find_and_plot_cluster_centers(ii)
    plt.title(ii)
    
    variances[ii - 1] = kmeans.inertia_

In [ ]:
variances

In [ ]:
from matplotlib.ticker import MaxNLocator

In [ ]:
plt.plot(np.arange(1, max_clusters_nb), variances)
plt.xlabel("Number of clusters")
plt.ylabel("Variance")
plt.title("Variance in function of the number of clusters")
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

#plt.yscale('log')

6 seems to be a good choice.

At 8, purple appears. It is a well represented color so we will go with this for now.
Once the entire pipeline will be up, we will try again different numbers of clsuters and compare the average precision.

In [ ]:
%%time
kmeans, color_lut = find_and_plot_cluster_centers(8)
color_lut

### 1.3. K-Means clustering stage 2/2: projection
The [`KMeans` class](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) provided by scikit-learn has two methods for transforming our data:
- `transform`
- `predict`

**Make sure you understand the difference between those two functions.**

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Use the `KMeans` object and super fancy Numpy indexing to create a new image where the color of each pixel is the color of the closest cluster to the original pixel color.**

Tip:
- Do not forget to mask the background of the image during prediction (because we did not train the predictor on them).
- Start by producing a new image with cluster labels instead of color values.
- Then create a color lookup table (LUT) using cluster centers (make sure to use `np.uint8` values to avoid issues with later conversion).
- Finally use Numpy advanced indexing to apply the LUT to the image with cluster labels.
</div>

In [ ]:
def recolor_masked(symbol, clusterer, color_lut):
    symbol_mask = symbol2mask(symbol)
    recolored = np.ones_like(symbol) * 255
    recolored[symbol_mask] = color_lut[np.uint8(clusterer.predict(symbol[symbol_mask]))]
    return recolored

In [ ]:
def recolor_masked_know(symbol, symbol_mask, clusterer, color_lut):
    recolored = np.ones_like(symbol) * 255
    recolored[symbol_mask] = color_lut[np.uint8(clusterer.predict(symbol[symbol_mask]))]
    return recolored

In [ ]:
def recolor_unmasked(image, clusterer, color_lut):
    return color_lut[np.uint8(clusterer.predict(image.reshape((image.shape[0] * image.shape[1], image.shape[2]))))].reshape((image.shape[0], image.shape[1], image.shape[2]))

In [ ]:
# Now create a recolored image `test_recolored` containing only indexed colors (ie. in the LUT)
# using the LUT `color_lut`
test_recolored = recolor_unmasked(reference_img, kmeans, color_lut)

In [ ]:
# let us display zoomed portions of the images
plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(reference_img)
plt.axis("off")
plt.title("Original")
plt.subplot(1,2,2)
plt.imshow(recolor_unmasked(reference_img, kmeans, color_lut))
plt.axis("off")
plt.title("Recolored")

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Does this look correct?**

</div>

Yes, thanks to MiniBatchKmeans.

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Save the image and compare its size with the original one. Make sure you use the right color space when saving!**

Tips:

- If saving the RGB image with OpenCV `cv2.imwrite()`, do not forget it expects a BGR image! 
</div>

In [ ]:
# save the image and compare the sizes
cv2.imwrite("/tmp/recolored.png", test_recolored[:,:,::-1])  # make sure the exported image is in BGR format here!
# Let us use some notebook magic to check the size of the image
!du -h /tmp/recolored.png
!du -h $PATH_TO_RESOURCES/dobble_ref.png

### 1.4. Load and convert all the bubbles
We can now load and convert all the bubbles. We need to:
1. load them
2. convert them to the same color space as use during training (RGB here)
3. project their colors using the previous method

We will need to ignore the area where all the pixels are black `(0,0,0)`, because they do not belong to the bubble and it may change their value.

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Now load all the bubbles and convert them to RGB (or any color space your choosed).**
</div>

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**And reduce the color of all bubbles.**
</div>

In [ ]:
%%time
# reduce the color of all bubbles
# This is exactly what we did for the full poster, but applied to each bubble AND keeping background as black.
symbols_quant = [recolor_masked_know(symbols[i], symbols_masks[i], kmeans, color_lut) for i in range(len(symbols))]

In [ ]:
# let us display the images
plt.figure(figsize=(8,16 * 5))
for ii in range(57):
    plt.subplot(57,2,1+2*ii)
    plt.imshow(symbols[ii * 5])
    plt.axis("off")
    if ii == 0:
        plt.title("Original")
    plt.subplot(57,2,2+2*ii)
    plt.imshow(symbols_quant[ii * 5])
    plt.axis("off")
    if ii == 0:
        plt.title("Recolored")

## 2. Compute the color histograms
To compute the color histogram of a bubble, we do not need to recolorize it, we just need to compute its "label map" to count the number of pixel belonging to each cluster.

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Compute the color histogram for each bubble. Use [`np.bincount`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.bincount.html#numpy.bincount) to count the number of number of occurence of each cluster label. Do not forget to normalize the histogram using the number of non-zero values in the mask with [`np.count_nonzero`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.count_nonzero.html).**
</div>

In [ ]:
def symbol2hist(symbol, symbol_mask, clusterer):
    return np.bincount(np.uint8(clusterer.predict(symbol[symbol_mask])), minlength = clusterer.n_clusters) / np.count_nonzero(symbol_mask)

In [ ]:
# compute the color histogram for each bubble (number of pixel in each cluster) then normalize it.
# `color_histograms` is a list of np.array, 
# each element being the histogram corresponding to a bubble in `bubbles`.
color_histograms = [symbol2hist(symbols[i], symbols_masks[i], kmeans) for i in range(len(symbols))]

In [ ]:
# Show some color histograms with colors!
colors_for_bars = color_lut/255  # matplotlib colors are RGB values scaled to [0,1]
for ii in range(57):
    plt.figure(figsize=(8,4))
    plt.subplot(1,3,1)
    symbol_index = ii * 5
    plt.imshow(symbols[symbol_index])
    plt.axis('off'); plt.title("Original")
    plt.subplot(1,3,2)
    plt.imshow(symbols_quant[symbol_index])
    plt.axis('off'); plt.title("Recolored")
    plt.subplot(1,3,3, aspect=len(color_histograms[0]))
    plt.ylim(0, 1)
    plt.bar(range(len(color_histograms[symbol_index])), 
            color_histograms[symbol_index]/color_histograms[symbol_index].max(), 
            color=colors_for_bars)
    plt.xticks([]); plt.title("Histogram")
    plt.show()

## 3. Compute the distance matrix
Because color histograms are **very compact**, it is **very fast** to compute the distance matrix (even if the complexity is $O(n^2)$).

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Compute the distance matrix between each pair of bubbles. Use an appropriate distance from [`scipy.spatial.distance`](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html).**
</div>

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
%%time
# compute the distance matrix between each pair of bubbles
dist_mat = np.ones((len(symbols), len(symbols)))  # distances will be between 0 (closest) and 1 (farthest)
for i in range(len(symbols)):
    for j in range(len(symbols)):
        dist_mat[i,j] = cosine(color_histograms[i], color_histograms[j])

Correct the diagonal to avoid getting the same result over and over: we set the distance of one element against itself to the maximum distance.

In [ ]:
dist_mat[np.diag_indices_from(dist_mat)] = 1

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Display the best matches for some (all?) bubbles, ie the bubbles which are the closest to a given one. Use `np.argsort` along the appropriate axis to get the indices of the closest elements along each line.**
</div>

In [ ]:
# Compute the indices of elements with smallest distance to each bubble (ie. in a matrix row)
# `idx_of_best_matches_per_row` is a 2D np.array: 
# for each row, the columns indicate the indices of the elements in `dist_mat` to sort the row (ascending order)
idx_of_best_matches_per_row = np.argsort(dist_mat)

In [ ]:
# Display the best matches for some symbols
max_res = 5
interesting_symbols_ids = [i * 5 for i in range(57)]

for ii in interesting_symbols_ids:
    plt.figure(figsize=(12,8))
    columns = max_res + 1
    plt.subplot(1, columns, 1)
    plt.imshow(symbols[ii])
    plt.axis("off"); plt.title("Symbol %d"%(ii,))
    for jj in range(max_res):
        bb_idx = idx_of_best_matches_per_row[ii, jj]  # Read the id of each best match for current symbol
        plt.subplot(1, columns, jj+2)
        plt.imshow(symbols[bb_idx])
        plt.axis("off"); plt.title("b%d@%.3f" % (bb_idx, dist_mat[ii, bb_idx])) # display symbol id and dist.
    plt.show()

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Write some notes about the advantages and the limitations of the color histogram approach.**
</div>

It does not use the form at all.

## 4. Hierarchical clustering and dendrograms
Instead of computing a distance matrix, it is also possible to aggregate the elements starting by the closest one, then iterating. The trick is to be able to compute the distance between a cluster and another cluster, and a simple solution is to average the descriptor of two clusters to form the descriptor of a new parent cluster.

The code below does all this work and produces a dendrogram.

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

**Call the functions (2nd and 3rd cells below) appropriately to generate a beautiful image.**
</div>

In [ ]:
# Adapted from https://github.com/jesolem/PCV/blob/master/PCV/clustering/hcluster.py
# licensed under the BSD 2-Clause "Simplified" License
from itertools import combinations

class ClusterNode(object):
    def __init__(self,vec,left,right,distance=0.0,count=1):
        self.left = left
        self.right = right
        self.vec = vec
        self.distance = distance
        self.count = count # only used for weighted average

    def extract_clusters(self,dist):
        """ Extract list of sub-tree clusters from 
            hcluster tree with distance<dist. """
        if self.distance < dist:
            return [self]
        return self.left.extract_clusters(dist) + self.right.extract_clusters(dist)

    def get_cluster_elements(self):
        """    Return ids for elements in a cluster sub-tree. """
        return self.left.get_cluster_elements() + self.right.get_cluster_elements()

    def get_height(self):
        """    Return the height of a node, 
            height is sum of each branch. """
        return self.left.get_height() + self.right.get_height()

    def get_depth(self):
        """    Return the depth of a node, depth is 
            max of each child plus own distance. """
        return max(self.left.get_depth(), self.right.get_depth()) + self.distance

    def draw(self,draw,x,y,s,imlist,im):
        """    Draw nodes recursively with image 
            thumbnails for leaf nodes. """
        h1 = int(self.left.get_height()*60 / 2)
        h2 = int(self.right.get_height()*60 /2)
        top = y-(h1+h2)
        bottom = y+(h1+h2)
        
        # vertical line to children
        cv2.line(draw, (int(top+h1), int(x)) , (int(bottom-h2),int(x)), (0,0,0))
        
        # horizontal lines 
        ll = self.distance*s
        cv2.line(draw, (int(top+h1),int(x)) , (int(top+h1), int(x+ll)), (0,0,0))
        cv2.line(draw, (int(bottom-h2), int(x)) , (int(bottom-h2), int(x+ll)), (0,0,0))
        
        # draw left and right child nodes recursively    
        self.left.draw(draw,x+ll,top+h1,s,imlist,im)
        self.right.draw(draw,x+ll,bottom-h2,s,imlist,im)
    

class ClusterLeafNode(object):
    def __init__(self,vec,id):
        self.vec = vec
        self.id = id

    def extract_clusters(self,dist):
        return [self] 

    def get_cluster_elements(self):
        return [self.id]

    def get_height(self):
        return 1

    def get_depth(self):
        return 0
    
    def draw(self,draw,x,y,s,imlist,im):
        nodeim = cv2.resize(imlist[self.id], (60,60))
        ns = nodeim.shape
        im[int(x):int(x+ns[0]), int(y-ns[1]//2):int(y+ns[1]-ns[1]//2), ...] = nodeim


def hcluster(features, distfcn):
    """
    Cluster the rows of features using hierarchical clustering.
    
    `features`: square matrix of features
    `distfcn`: distance function to compare features (feature x feature -> float)
    """
    
    # cache of distance calculations
    distances = {}
    
    # initialize with each row as a cluster 
    node = [ClusterLeafNode(f,id=i) for i,f in enumerate(features)]
    
    while len(node)>1:
        closest = float('Inf')
        
        # loop through every pair looking for the smallest distance
        for ni,nj in combinations(node,2):
            if (ni,nj) not in distances: 
                distances[ni,nj] = distfcn(ni.vec,nj.vec)
                
            d = distances[ni,nj]
            if d<closest:
                closest = d 
                lowestpair = (ni,nj)
        ni,nj = lowestpair
        
        # average the two clusters
        new_vec = (ni.vec + nj.vec) / 2.0
        
        # create new node
        new_node = ClusterNode(new_vec,left=ni,right=nj,distance=closest)
        node.remove(ni)
        node.remove(nj)
        node.append(new_node)
    
    return node[0]

 
def draw_dendrogram(node, imlist, filename='dendrogram.jpg'):
    """
    Draw a cluster dendrogram and save to a file. 
    
    `node`: result of the `hcluster` function
    `imlist`: list of BGR images for each element
    `filename`: path to output file
    """
    # height and width
    cols = node.get_height()*60
    rows = 1200

    # scale factor for distances to fit image width
    s = float(rows-150)/node.get_depth()
    
    # create image and draw object
    im = np.full((rows,cols, 3), 255, dtype=np.uint8)
    draw = im
    
    # initial line for start of tree 
    cv2.line(draw, (cols//2, 0),(cols//2, 60), (0,0,0))
    
    # draw the nodes recursively
    node.draw(draw,60,(cols/2),s,imlist,im)
    cv2.imwrite(filename, draw)
    print("Wrote '%s'." % filename)

In [ ]:
# generate the clusters using the features you computed, and a distance function of your choice
cluster = hcluster(dist_mat, cosine)

In [ ]:
symbols_bgr = [cv2.cvtColor(symbol, cv2.COLOR_RGB2BGR) for symbol in symbols]
draw_dendrogram(cluster, symbols_bgr, "../resources/my_dendrogram.jpg")

# Job done!
Do not forget to submit your notebooks (and maybe a scaled version of your dendrogram)!